# 120. User-Based Collaborative Filtering (협업 필터링)

### User-Based CF 의 알고리즘

| USER | M1 | M2 | M3 | M4 | M5 | U1과의 유사도 |
| --- | --- |--- |--- | ---|--- | --- |
| U1 | 2 | 5 | 3 |   |   |
|U2|4|4|3|5|1|0.19|
|U3|1|5|4| | 5| 0.89|
|U4|3|5|3|2|5  |0.94|
|U5|4|5|3|4| |0.65|
|U3, U4 평균|2|5|3.5|2|5|

1) U1과 취향이 비슷한 사용자를 찾는다. 취향은 각 사용자의 영화에 대한 평가의 유사성을 계산하여 찾는다. .U3와 U4가 U1과 가장 높은 상관 관계를 보임.  

2) U1 과 가장 유사한 U3와 U4가 가장 좋게 평가한 영화를 찾는다. U1 이 아직 보지 않은 영화 M4, M5 에 대해 U3, U4 의 평점 평균을 내면 각각 2 와 5 이다. 따라서 평점 평균이 높은 M5를 U1도 좋아할 것으로 예상.

### MovieLens dataset
- MovieLens 라는 테스트 추천 시스템으로부터 수집된 데이터  
- 사용자들이 영화에 대해서 평가한 데이터  
- MovieLens에서는 사용자가 각 영화를 1점(최악)에서 5점(최고) 사이의 점수로 평가  

- content-based filtering 과 달리 CF 는 similarity 를 이용하므로 **item의 feature 정보가 필요하지 않기 때문**에 movies dataframe에서 title 을 제외한 모든 다른 열을 drop 한다.  

- ratings 만을 입력으로 사용

### train 데이터로 평점 matrix 생성

### User-Based CF 알고리즘

1) 모든 사용자간의 평가의 유사도를 계산  
2) 현재 추천 대상이 되는 사용자와 다른 사용자의 유사도를 추출  
3) 현재 사용자가 평가하지 않은 모든 아이템에 대해서 현재 사용자의 예상 평가값 계산. 예상 평가값은 다른 사용자의 해당 아이템에 대한 평가(평점)를 현재 사용자와 그 사용자의 유사도로 가중 평균  
4) 아이템 중에서 예상 평가값이 가장 높은 N 개의 아이템을 추천  

이 알고리즘이 작동하기 위해서는 추천 대상 사용자가 평가한 아이템과 다른 사용자들이 평가한 동일한 아이템이 유사하게 계산되면 신뢰할 수 있는 모델이 만들어진 것이고, 이 모델을 이용하여 추천 대상 사용자가 보지 않은 아이템(영화)의 평점을 계산하고 가장 예상 평점이 높은 N 개 아이템을 추천하면 된다.    

이때, 추천 대상 사용자와 가장 유사도가 높은 K 개의 다른 사용자의 평가만 반영하여 예측 평가의 품질을 높인다.

이를 위해 추천 사용자, 영화 id 를 parameter 로 받아 예상 평점을 반환하는 함수를 작성

### 추천 대상이 되는 사용자에게 보지 않은 영화를 추천할 때 사용자가 부여할 rating 예측

1. 추천 대상 영화에 대한 모든 사용자의 rating값 가져오기  
2. 추천 대상 영화를 평가하지 않은 사용자의 index 가져오기
3. 대상 영화를 평가하지 않은 사용자의 rating (null) 제거 --> 추천 대상 영화를 평가한 사용자만 남김

- ratings_matrix 에서 user_id 1 이 movie_id 1 을 평가하지 않았으므로 user_id 1 에게 movie_id 1을 추천할 때의 예상 rating을 예측해 봅니다.

- 현재 추천 대상이 되는 사용자와 다른 사용자의 유사도를 추출

### 가장 유사도가 높은 상위 K명의 사용자 찾기

### Implicit Rating을 이용한 추천 공식

$$\tilde{r}_{ui} = \frac{\sum_{v \in KNN(u)}r_{vi} \cdot S_{vu}}{\sum_{v \in KNN(u)}S_{vu}}$$

###  특정 사용자에게 추천하는 영화(movie_id)의  가중평균 rating을 계산하는 함수
- 위의 과정을 model 함수로 작성
- 위에서 개별 사용자와 아이템에 대해 구하던 predicted_rating을 함수로 작성
- 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)

### 모델 성능 평가 - 모델별 RMSE를 계산하는 함수 
- Test set을 이용하여 모델 성능 측정  

### 특정 사용자에 대하여 영화 추천

영화를 예상 평점에 따라 정렬해서 제목을 뽑아 줌

### 주어진 사용자에 대하여 추천 받는 함수
위 코드들을 하나의 함수로 작성 

### 최적의 neighbor size 구하기